## Week 5 - Final Assignment

##### Description and Details about the project are here:
https://github.com/LRT-YASH/TestDSRepo/blob/master/WK4-Problem%20and%20Dataset%20Information.txt

### Plan of execution:
Step 1: Import the datasets from CSV as they are available in CSV, remove not needed attributes, merge some attributes as per need and changes cases/types

Step 2: Connect to foursquare and import venues data like Mexican restuarants by community and their price


In [40]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')


Libraries imported.


In [41]:
!pip install xlrd

In [42]:
Chicago_Neigh_Demo = pd.read_csv('Demographics_Chicago_2008_2012.csv')
Chicago_Neigh_Demo

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,14.0,Albany Park,11.3,19.2,10.0,32.9,32.0,21323,53.0
1,57.0,Archer Heights,8.5,14.1,16.5,35.9,39.2,16134,67.0
2,34.0,Armour Square,5.7,40.1,16.7,34.5,38.3,16148,82.0
3,70.0,Ashburn,4.0,10.4,11.7,17.7,36.9,23482,37.0
4,71.0,Auburn Gresham,4.0,27.6,28.3,18.5,41.9,15528,74.0
...,...,...,...,...,...,...,...,...,...
73,65.0,West Lawn,5.8,14.9,9.6,33.6,39.6,16907,56.0
74,53.0,West Pullman,3.3,25.9,19.4,20.5,42.1,16563,62.0
75,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
76,24.0,West Town,2.3,14.7,6.6,12.9,21.7,43198,10.0


In [43]:
import pandas as pd

Chicago_Neigh_Data = pd.read_excel (r'Chicago_Neighborhoods.xlsx') #for an earlier version of Excel, you may need to use the file extension of 'xls'
Chicago_Neigh = pd.DataFrame(Chicago_Neigh_Data, columns = ['Geog','GeogKey','Total Population','Not Hispanic or Latino but White alone','Not Hispanic or Latino but Black or African American alone','Not Hispanic or Latino but American Indian and Alaska Native alone','Not Hispanic or Latino but Asian alone','Not Hispanic or Latino but Native Hawaiian and Other Pacific Islander alone','Not Hispanic or Latino but Some Other Race alone','Not Hispanic or Latino but Two or More Races','Hispanic or Latino','Male:  to 5 yrs','Male: 5 to 9 yrs','Male: 10 to 14 yrs','Male: 15 to 17 yrs','Male: 18 and 19 yrs','Male: 20 yrs','Male: 21 yrs','Male: 22 to 24 yrs','Male: 25 to 29 yrs','Male: 30 to 34 yrs','Male: 35 to 39 yrs','Male: 40 to 44 yrs','Male: 45 to 49 yrs','Male: 50 to 54 yrs','Male: 55 to 59 yrs','Male: 60 to 61 yrs','Male: 62 to 64 yrs','Male: 65 to 66 yrs','Male: 67 to 69 yrs','Male: 70 to 74 yrs','Male: 75 to 79 yrs','Male: 80 to 84 yrs','Male: 85 yrs and over','FeMale: Under 5 yrs','FeMale: 5 to 9 yrs','FeMale: 10 to 14 yrs','FeMale: 15 to 17 yrs','FeMale: 18 to 19 yrs','FeMale: 20 yrs','FeMale: 21 yrs','FeMale: 22 to 24 yrs','FeMale: 25 to 29 yrs','FeMale: 30 to 34 yrs','FeMale: 35 to 39 yrs','FeMale: 40 to 44 yrs','FeMale: 45 to 49 yrs','FeMale: 50 to 54 yrs','FeMale: 55 to 59 yrs','FeMale: 60 to 61 yrs','FeMale: 62 to 64 yrs','FeMale: 65 to 66 yrs','FeMale: 67 to 69 yrs','FeMale: 70 to 74 yrs','FeMale: 75 to 79 yrs','FeMale: 80 to 84 yrs','FeMale: 85 yrs and over','Median Age','Total Households'])
Chicago_Neigh.head(5)

,Geog,GeogKey,Total Population,Not Hispanic or Latino but White alone,Not Hispanic or Latino but Black or African American alone,Not Hispanic or Latino but American Indian and Alaska Native alone,Not Hispanic or Latino but Asian alone,Not Hispanic or Latino but Native Hawaiian and Other Pacific Islander alone,Not Hispanic or Latino but Some Other Race alone,Not Hispanic or Latino but Two or More Races,...,FeMale: 60 to 61 yrs,FeMale: 62 to 64 yrs,FeMale: 65 to 66 yrs,FeMale: 67 to 69 yrs,FeMale: 70 to 74 yrs,FeMale: 75 to 79 yrs,FeMale: 80 to 84 yrs,FeMale: 85 yrs and over,Median Age,Total Households
0,Rogers Park,1,54991,21618,14461,142,3523,39,202,1573,...,486,661,308,383,534,392,312,388,32.404941,24292
1,West Ridge,2,71942,30706,8015,189,16184,3,225,1919,...,794,1026,561,718,1056,799,713,931,35.062344,24755
2,Uptown,3,56362,29098,11275,191,6414,10,130,1235,...,561,679,360,475,763,620,524,690,35.464521,29719
3,Lincoln Square,4,39493,24916,1488,72,4399,11,128,917,...,322,493,256,323,436,340,275,316,33.450642,18194
4,North Center,5,31867,24634,738,55,1449,16,51,594,...,261,314,159,218,319,257,181,203,32.478928,14561


In [44]:
# Cleaning up of the datasets - Demographics data
##Cleaning up includes, removing those age groups that eat in a restuarant very less, ages < 15yrs and < 80yrs
##Male Cleaning up includes, merging some columns and dropping after merge 
Chicago_Neigh.drop(['Male:  to 5 yrs','Male: 5 to 9 yrs','Male: 10 to 14 yrs'],axis = 1, inplace = True)
Chicago_Neigh['Male: 15 to 70 yrs'] = Chicago_Neigh['Male: 15 to 17 yrs'] + Chicago_Neigh['Male: 18 and 19 yrs'] + Chicago_Neigh['Male: 20 yrs'] + Chicago_Neigh['Male: 21 yrs'] + Chicago_Neigh['Male: 22 to 24 yrs'] + Chicago_Neigh['Male: 25 to 29 yrs'] + Chicago_Neigh['Male: 30 to 34 yrs'] + Chicago_Neigh['Male: 35 to 39 yrs'] + Chicago_Neigh['Male: 40 to 44 yrs'] + Chicago_Neigh['Male: 45 to 49 yrs'] + Chicago_Neigh['Male: 50 to 54 yrs'] + Chicago_Neigh['Male: 55 to 59 yrs'] + Chicago_Neigh['Male: 60 to 61 yrs']+ Chicago_Neigh['Male: 62 to 64 yrs'] + Chicago_Neigh['Male: 65 to 66 yrs'] + Chicago_Neigh['Male: 67 to 69 yrs']
Chicago_Neigh.drop(['Male: 15 to 17 yrs','Male: 18 and 19 yrs','Male: 20 yrs','Male: 21 yrs','Male: 22 to 24 yrs','Male: 25 to 29 yrs','Male: 30 to 34 yrs','Male: 35 to 39 yrs','Male: 40 to 44 yrs','Male: 45 to 49 yrs','Male: 50 to 54 yrs','Male: 55 to 59 yrs','Male: 70 to 74 yrs','Male: 75 to 79 yrs','Male: 80 to 84 yrs','Male: 60 to 61 yrs','Male: 62 to 64 yrs','Male: 65 to 66 yrs','Male: 67 to 69 yrs','Male: 85 yrs and over'],axis = 1, inplace = True)
##Female Cleaning up includes, merging some columns and dropping after merge 
Chicago_Neigh.drop(['FeMale: Under 5 yrs','FeMale: 5 to 9 yrs','FeMale: 10 to 14 yrs','FeMale: 70 to 74 yrs','FeMale: 75 to 79 yrs','FeMale: 80 to 84 yrs','FeMale: 85 yrs and over'],axis = 1, inplace = True)
Chicago_Neigh['Female: 15 to 70 yrs'] = Chicago_Neigh['FeMale: 15 to 17 yrs']+Chicago_Neigh['FeMale: 18 to 19 yrs']+Chicago_Neigh['FeMale: 20 yrs']+Chicago_Neigh['FeMale: 21 yrs']+Chicago_Neigh['FeMale: 22 to 24 yrs']+Chicago_Neigh['FeMale: 25 to 29 yrs']+Chicago_Neigh['FeMale: 30 to 34 yrs']+Chicago_Neigh['FeMale: 35 to 39 yrs']+Chicago_Neigh['FeMale: 40 to 44 yrs']+Chicago_Neigh['FeMale: 45 to 49 yrs']+Chicago_Neigh['FeMale: 50 to 54 yrs']+Chicago_Neigh['FeMale: 55 to 59 yrs']+Chicago_Neigh['FeMale: 60 to 61 yrs']+Chicago_Neigh['FeMale: 62 to 64 yrs']+Chicago_Neigh['FeMale: 65 to 66 yrs']+Chicago_Neigh['FeMale: 67 to 69 yrs']
Chicago_Neigh.drop(['FeMale: 15 to 17 yrs','FeMale: 18 to 19 yrs','FeMale: 20 yrs','FeMale: 21 yrs','FeMale: 22 to 24 yrs','FeMale: 25 to 29 yrs','FeMale: 30 to 34 yrs','FeMale: 35 to 39 yrs','FeMale: 40 to 44 yrs','FeMale: 45 to 49 yrs','FeMale: 50 to 54 yrs','FeMale: 55 to 59 yrs','FeMale: 60 to 61 yrs','FeMale: 62 to 64 yrs','FeMale: 65 to 66 yrs','FeMale: 67 to 69 yrs'],axis = 1, inplace = True)
## Merging all communities
Chicago_Neigh['Not Hispanic or Latino'] = Chicago_Neigh['Not Hispanic or Latino but White alone']+Chicago_Neigh['Not Hispanic or Latino but Black or African American alone']+Chicago_Neigh['Not Hispanic or Latino but American Indian and Alaska Native alone']+Chicago_Neigh['Not Hispanic or Latino but Asian alone']+Chicago_Neigh['Not Hispanic or Latino but Native Hawaiian and Other Pacific Islander alone']+Chicago_Neigh['Not Hispanic or Latino but Some Other Race alone']+Chicago_Neigh['Not Hispanic or Latino but Two or More Races']
Chicago_Neigh.drop(['GeogKey','Not Hispanic or Latino but White alone','Not Hispanic or Latino but Black or African American alone','Not Hispanic or Latino but American Indian and Alaska Native alone','Not Hispanic or Latino but Asian alone','Not Hispanic or Latino but Native Hawaiian and Other Pacific Islander alone','Not Hispanic or Latino but Some Other Race alone','Not Hispanic or Latino but Two or More Races'],axis = 1, inplace = True)
Chicago_Neigh.rename(columns={'Geog':'COMMUNITY NAME'}, inplace=True)
Chicago_Neigh['COMMUNITY NAME'] =  Chicago_Neigh['COMMUNITY NAME'].str.upper()
Chicago_Neigh['Hispanic or Lation % of Total'] = Chicago_Neigh['Hispanic or Latino']/Chicago_Neigh['Total Population'] * 100
Chicago_Neigh.shape

(77, 9)

In [45]:
Chicago_Neigh_Demo.rename(columns={'COMMUNITY AREA NAME':'COMMUNITY NAME'}, inplace=True)
Chicago_Neigh_Demo.drop(['Community Area Number','PERCENT OF HOUSING CROWDED','PERCENT HOUSEHOLDS BELOW POVERTY','PERCENT AGED 16+ UNEMPLOYED','PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA','HARDSHIP INDEX'],axis=1, inplace=True)
Chicago_Neigh_Demo['COMMUNITY NAME'] =  Chicago_Neigh_Demo['COMMUNITY NAME'].str.upper()
Chicago_Neigh_Demo.set_index("COMMUNITY NAME")
Chicago_Neigh_Demo.reset_index()
print(Chicago_Neigh_Demo.dtypes)
Chicago_Neigh_Demo.tail()

COMMUNITY NAME                       object
PERCENT AGED UNDER 18 OR OVER 64    float64
PER CAPITA INCOME                     int64
dtype: object


,COMMUNITY NAME,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME
73,WEST LAWN,39.6,16907
74,WEST PULLMAN,42.1,16563
75,WEST RIDGE,38.5,23040
76,WEST TOWN,21.7,43198
77,WOODLAWN,36.1,18672


In [46]:
#Import GeoCodes CSV
Chicago_geocode = pd.read_csv('Chicago_Lat_Long.csv')
Chicago_geocode.rename(columns={'Community Name':'COMMUNITY NAME'}, inplace=True)
Chicago_geocode['COMMUNITY NAME'] =  Chicago_geocode['COMMUNITY NAME'].str.upper()
## In this case we are merging both result sets to make it ready for 
Chicago_Neigh_stg = pd.merge(Chicago_Neigh_Demo, Chicago_Neigh, on='COMMUNITY NAME', how='outer')
Chicago_Neigh_Full = pd.merge(Chicago_Neigh_stg, Chicago_geocode, on='COMMUNITY NAME', how='outer')
Chicago_Neigh_Full.set_index('COMMUNITY NAME')
# Chicago_Neigh_Full.sort_values(['Hispanic or Lation % of Total','PER CAPITA INCOME ','PERCENT AGED UNDER 18 OR OVER 64'],ascending=False,inplace=True) 
Chicago_Neigh_Full.head(35)

,COMMUNITY NAME,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,Total Population,Hispanic or Latino,Median Age,Total Households,Male: 15 to 70 yrs,Female: 15 to 70 yrs,Not Hispanic or Latino,Hispanic or Lation % of Total,Latitude,Longitude
0,ALBANY PARK,32.0,21323,51542.0,25487.0,31.078026,16322.0,19715.0,18065.0,26055.0,49.448993,41.961451,-87.696929
1,ARCHER HEIGHTS,39.2,16134,13393.0,10182.0,30.507134,3844.0,4734.0,4483.0,3211.0,76.024789,41.807749,-87.722579
2,ARMOUR SQUARE,38.3,16148,13391.0,464.0,41.118083,5247.0,4725.0,4902.0,12927.0,3.465014,41.853035,-87.630782
3,ASHBURN,36.9,23482,41081.0,15132.0,33.483019,12451.0,13771.0,14939.0,25949.0,36.834546,41.749095,-87.679637
4,AUBURN GRESHAM,41.9,15528,48743.0,459.0,36.969525,17603.0,14402.0,18888.0,48284.0,0.941674,41.752451,-87.652969
5,AUSTIN,37.9,15957,98514.0,8722.0,33.191489,32792.0,31618.0,38297.0,89792.0,8.853564,41.882286,-87.764980
6,AVALON PARK,39.3,24454,10185.0,153.0,42.892560,3924.0,3027.0,3995.0,10032.0,1.502209,41.736969,-87.587714
7,AVONDALE,31.0,20039,39262.0,25295.0,31.156746,13573.0,15209.0,13880.0,13967.0,64.426163,41.937633,-87.693190
8,BELMONT CRAGIN,37.3,15461,78743.0,62101.0,30.292657,21591.0,28249.0,27115.0,16642.0,78.865423,41.916967,-87.782607
9,BEVERLY,40.5,39523,20034.0,915.0,40.889807,7546.0,6792.0,7477.0,19119.0,4.567236,41.711666,-87.681963


In [51]:
Chicago_Neigh_Full['Income_Rank'] = Chicago_Neigh_Full['PER CAPITA INCOME '].rank(ascending = 0) 
Chicago_Neigh_Full['H&L_Pop%_Rank'] = Chicago_Neigh_Full['Hispanic or Lation % of Total'].rank(ascending = 0) 
Chicago_Neigh_Full['Age_Rank'] = Chicago_Neigh_Full['PERCENT AGED UNDER 18 OR OVER 64'].rank(ascending = 0) 
Chicago_Neigh_Full

,COMMUNITY NAME,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,Total Population,Hispanic or Latino,Median Age,Total Households,Male: 15 to 70 yrs,Female: 15 to 70 yrs,Not Hispanic or Latino,Hispanic or Lation % of Total,Latitude,Longitude,Income_Rank,H&L_Pop%_Rank,Age_Rank,Income_&_Hispanic_Pop_Combined_Rank
0,ALBANY PARK,32.0,21323,51542.0,25487.0,31.078026,16322.0,19715.0,18065.0,26055.0,49.448993,41.961451,-87.696929,39.0,18.0,58.0,58
1,ARCHER HEIGHTS,39.2,16134,13393.0,10182.0,30.507134,3844.0,4734.0,4483.0,3211.0,76.024789,41.807749,-87.722579,56.0,10.0,31.0,67
2,ARMOUR SQUARE,38.3,16148,13391.0,464.0,41.118083,5247.0,4725.0,4902.0,12927.0,3.465014,41.853035,-87.630782,55.0,58.0,38.0,114
3,ASHBURN,36.9,23482,41081.0,15132.0,33.483019,12451.0,13771.0,14939.0,25949.0,36.834546,41.749095,-87.679637,33.0,24.0,45.0,58
4,AUBURN GRESHAM,41.9,15528,48743.0,459.0,36.969525,17603.0,14402.0,18888.0,48284.0,0.941674,41.752451,-87.652969,59.0,75.0,14.0,135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,WEST LAWN,39.6,16907,33355.0,26669.0,30.000717,8766.0,11550.0,11203.0,6686.0,79.955029,41.777732,-87.713878,51.0,6.0,25.0,58
74,WEST PULLMAN,42.1,16563,29651.0,1509.0,33.928071,9492.0,9190.0,11370.0,28142.0,5.089204,41.672371,-87.649842,53.0,52.0,13.0,106
75,WEST RIDGE,38.5,23040,71942.0,14701.0,35.062344,24755.0,25321.0,25232.0,57241.0,20.434517,41.992071,-87.675487,35.0,31.0,37.0,67
76,WEST TOWN,21.7,43198,82236.0,23901.0,31.257042,38190.0,34955.0,32724.0,58335.0,29.063914,41.899386,-87.696928,9.0,26.0,74.0,35


In [52]:
Chicago_Neigh_Full = Chicago_Neigh_Full.fillna(0)
Chicago_Neigh_Full['Income_Rank'] = Chicago_Neigh_Full['Income_Rank'].astype(int)
Chicago_Neigh_Full['H&L_Pop%_Rank'] = Chicago_Neigh_Full['H&L_Pop%_Rank'].astype(int)
Chicago_Neigh_Full['Age_Rank'] = Chicago_Neigh_Full['Age_Rank'].astype(int)
Chicago_Neigh_Full.where(Chicago_Neigh_Full['Income_Rank'] < 15)
Chicago_Neigh_Full['Income_&_Hispanic_Pop_Combined_Rank'] = Chicago_Neigh_Full['Income_Rank']+Chicago_Neigh_Full['H&L_Pop%_Rank']
Chicago_Neigh_Full.dtypes
# Chicago_Neigh_Full.sort_values(['Income_Rank','H&L_Pop%_Rank','Age_Rank'],ascending=True)

COMMUNITY NAME                          object
PERCENT AGED UNDER 18 OR OVER 64       float64
PER CAPITA INCOME                        int64
Total Population                       float64
Hispanic or Latino                     float64
Median Age                             float64
Total Households                       float64
Male: 15 to 70 yrs                     float64
Female: 15 to 70 yrs                   float64
Not Hispanic or Latino                 float64
Hispanic or Lation % of Total          float64
Latitude                               float64
Longitude                              float64
Income_Rank                              int64
H&L_Pop%_Rank                            int64
Age_Rank                                 int64
Income_&_Hispanic_Pop_Combined_Rank      int64
dtype: object

In [53]:
#Chicago_Neigh_Full.reset_index("COMMUNITY NAME")
# Chicago_Neigh_Full
#Chicago_Neigh_Full.reset_index()
indexNames = Chicago_Neigh_Full[(Chicago_Neigh_Full["COMMUNITY NAME"] == "CHICAGO")].index
Chicago_Neigh_Full.drop(indexNames , inplace=True)
Chicago_Neigh_Full['Income_&_Hispanic_Pop_Combined_Rank'] = Chicago_Neigh_Full['Income_Rank']+Chicago_Neigh_Full['H&L_Pop%_Rank']
Chicago_Neigh_Top_List =  Chicago_Neigh_Full[(Chicago_Neigh_Full['Income_&_Hispanic_Pop_Combined_Rank'] < 50)]
Chicago_Neigh_Top_List

,COMMUNITY NAME,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,Total Population,Hispanic or Latino,Median Age,Total Households,Male: 15 to 70 yrs,Female: 15 to 70 yrs,Not Hispanic or Latino,Hispanic or Lation % of Total,Latitude,Longitude,Income_Rank,H&L_Pop%_Rank,Age_Rank,Income_&_Hispanic_Pop_Combined_Rank
17,CLEARING,37.6,25113,23139.0,10484.0,35.540070,8467.0,8090.0,8230.0,12655.0,45.308786,41.775271,-87.751894,28,20,43,48
25,FOREST GLEN,40.5,44164,18508.0,2126.0,42.522330,6864.0,6058.0,6405.0,16382.0,11.486925,41.990214,-87.733479,8,39,20,47
28,GARFIELD RIDGE,38.1,26353,34513.0,13515.0,37.743557,11927.0,11916.0,11875.0,20998.0,39.159157,41.785894,-87.738031,25,22,39,47
35,IRVING PARK,31.6,27249,53359.0,24332.0,33.276580,20376.0,20254.0,19653.0,29027.0,45.600555,41.959528,-87.698606,22,19,59,41
38,LAKE VIEW,17.0,60058,94368.0,7198.0,30.411351,54425.0,41137.0,40661.0,87170.0,7.627586,41.947370,-87.654465,4,45,76,49
40,LINCOLN SQUARE,25.5,37524,39493.0,7562.0,33.450642,18194.0,15451.0,16002.0,31931.0,19.147697,41.968913,-87.674871,13,33,68,46
41,LOGAN SQUARE,26.2,31908,72791.0,37295.0,30.710988,29352.0,28987.0,27991.0,35496.0,51.235730,41.917349,-87.666405,19,16,66,35
50,NEAR WEST SIDE,22.2,44689,54881.0,5048.0,30.608212,26298.0,22789.0,23302.0,49833.0,9.198083,41.881643,-87.647206,7,41,71,48
52,NORTH CENTER,26.2,57123,31867.0,4330.0,32.478928,14561.0,12172.0,12570.0,27537.0,13.587724,41.954342,-87.674439,6,37,66,43
76,WEST TOWN,21.7,43198,82236.0,23901.0,31.257042,38190.0,34955.0,32724.0,58335.0,29.063914,41.899386,-87.696928,9,26,74,35


In [55]:
## Adding Lat and Long information of all the communities
neighbourhood_latitude = Chicago_Neigh_Top_List.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = Chicago_Neigh_Top_List.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = Chicago_Neigh_Top_List.loc[0, 'COMMUNITY NAME'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

KeyError: 0

### Using Foursquare API to get the Neighbourhood Mexican restuarants and their price tag

In [56]:
CLIENT_ID = '0MWLXE21XWIGAFGOXET21G53OQXRSGUSCV5DFZWDTA5ESPJ5' # your Foursquare ID
CLIENT_SECRET = 'VPTKIT4VQCH5VDZS0OXN2MJIX2JE0LP1ZRADXFQTUUXE55ZS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0MWLXE21XWIGAFGOXET21G53OQXRSGUSCV5DFZWDTA5ESPJ5
CLIENT_SECRET:VPTKIT4VQCH5VDZS0OXN2MJIX2JE0LP1ZRADXFQTUUXE55ZS


In [57]:
LIMIT = 5
radius = 500
categoryId='4bf58dd8d48988d1c1941735'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighbourhood_latitude, neighbourhood_longitude, VERSION, categoryId, radius, LIMIT)
print(url)

https://api.foursquare.com/v2/venues/search?client_id=0MWLXE21XWIGAFGOXET21G53OQXRSGUSCV5DFZWDTA5ESPJ5&client_secret=VPTKIT4VQCH5VDZS0OXN2MJIX2JE0LP1ZRADXFQTUUXE55ZS&ll=41.961451000000004,-87.696929&v=20180605&categoryId=4bf58dd8d48988d1c1941735&radius=500&limit=5


In [58]:
!pip install geocoder
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # import geocoder
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [59]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
latitude = 41.961451
longitude = -87.6298
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, community in zip(Chicago_Neigh_Full['Latitude'], Chicago_Neigh_Full['Longitude'], Chicago_Neigh_Full['COMMUNITY NAME']):
    label = '{}'.format(community)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

In [ ]:
#results = requests.get(url).json()["response"]['groups'][0]['items']
#results

In [60]:
###Get the mexican restuarants 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId=4bf58dd8d48988d1c1941735&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            v['venue']['id'],
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['id',
                  'COMMUNITY NAME', 
                  'COMMUNITY Latitude', 
                  'COMMUNITY Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [62]:
Community_Mex_Rest = getNearbyVenues(names=Chicago_Neigh_Top_List['COMMUNITY NAME'],
                                   latitudes=Chicago_Neigh_Top_List['Latitude'],
                                   longitudes=Chicago_Neigh_Top_List['Longitude']
                                  )
Community_Mex_Rest

CLEARING
FOREST GLEN
GARFIELD RIDGE
IRVING PARK
LAKE VIEW
LINCOLN SQUARE
LOGAN SQUARE
NEAR WEST SIDE
NORTH CENTER
WEST TOWN


,id,COMMUNITY NAME,COMMUNITY Latitude,COMMUNITY Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,4ec0392e775b98876e62ca1d,GARFIELD RIDGE,41.785894,-87.738031,Salgados Grocery,41.786266,-87.734107,Mexican Restaurant
1,592d7bf10a464d4b2932e9fa,GARFIELD RIDGE,41.785894,-87.738031,DeColores,41.788128,-87.742233,Mexican Restaurant
2,5a29d170bed483458a1dbb21,GARFIELD RIDGE,41.785894,-87.738031,Des Colores,41.788956,-87.742392,Burrito Place
3,4bdc8a3a2a3a0f47cd50b3b6,IRVING PARK,41.959528,-87.698606,Que Rico,41.959259,-87.699812,Mexican Restaurant
4,517c6fe4e4b019c93a3978af,IRVING PARK,41.959528,-87.698606,Emma's Fruits @ Horner Park,41.959496,-87.698189,Mexican Restaurant
5,4b5169caf964a520c14c27e3,LAKE VIEW,41.947370,-87.654465,Taco & Burrito Palace,41.945653,-87.649464,Mexican Restaurant
6,4a7d1e95f964a52085ee1fe3,LAKE VIEW,41.947370,-87.654465,El Burrito Mexicano,41.947361,-87.653407,Mexican Restaurant
7,53a52919498ebd730dba2b1a,LAKE VIEW,41.947370,-87.654465,Salsita,41.949460,-87.649637,Mexican Restaurant
8,49ca69fff964a520a8581fe3,LAKE VIEW,41.947370,-87.654465,Las Mañanitas,41.946222,-87.649428,Mexican Restaurant
9,5ac2cd2f35811b76aae32b5f,LAKE VIEW,41.947370,-87.654465,Big Star,41.948286,-87.657419,Taco Place


In [ ]:
###Get the mexican restuarants 
def getPriceByVenues(id):
    
    venues_price_list=[]
    for id in id:
        print(id)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(id,CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
        print(url)
        # make the GET request
        results = requests.get(url).json()['response']['venue']['price']
        print(results)
        print('results complete')
        # return only relevant information for each nearby venue
        venues_price_list.append([(
            v['venue']['id']
            ,v['venue']['price']['message']
            ) for v in results])
        #print(venues_price_list)
    nearby_venues_list = pd.DataFrame([item for venues_price_list in venues_price_list for item in venues_price_list])
    nearby_venues_list.columns = ['id' #, 
                  'Price_Type' 
                            ]
    
    return(nearby_venues_list)

In [ ]:
#Chicago_venues_price = getPriceByVenues(id=Chicago_venues['id'])
#Chicago_Mex_rest = pd.merge(Chicago_venues, Chicago_venues_price, on='id', how='outer')

In [76]:
#Community_Mex_Rest.to_csv(r'Community_Mex_Rest.csv')
Community_Mex_Rest = pd.read_csv('Community_Mex_Rest.csv')
Community_Mex_Rest.drop(['Unnamed: 0'],axis=1, inplace = True)
Community_Mex_Rest

,id,COMMUNITY NAME,COMMUNITY Latitude,COMMUNITY Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Res_Tier
0,4ec0392e775b98876e62ca1d,GARFIELD RIDGE,41.785894,-87.738031,Salgados Grocery,41.786266,-87.734107,Mexican Restaurant,Cheap
1,592d7bf10a464d4b2932e9fa,GARFIELD RIDGE,41.785894,-87.738031,DeColores,41.788128,-87.742233,Mexican Restaurant,Cheap
2,5a29d170bed483458a1dbb21,GARFIELD RIDGE,41.785894,-87.738031,Des Colores,41.788956,-87.742392,Burrito Place,Cheap
3,4bdc8a3a2a3a0f47cd50b3b6,IRVING PARK,41.959528,-87.698606,Que Rico,41.959259,-87.699812,Mexican Restaurant,Cheap
4,517c6fe4e4b019c93a3978af,IRVING PARK,41.959528,-87.698606,Emma's Fruits @ Horner Park,41.959496,-87.698189,Mexican Restaurant,Cheap
5,4b5169caf964a520c14c27e3,LAKE VIEW,41.947370,-87.654465,Taco & Burrito Palace,41.945653,-87.649464,Mexican Restaurant,Cheap
6,4a7d1e95f964a52085ee1fe3,LAKE VIEW,41.947370,-87.654465,El Burrito Mexicano,41.947361,-87.653407,Mexican Restaurant,Cheap
7,53a52919498ebd730dba2b1a,LAKE VIEW,41.947370,-87.654465,Salsita,41.949460,-87.649637,Mexican Restaurant,Cheap
8,49ca69fff964a520a8581fe3,LAKE VIEW,41.947370,-87.654465,Las Mañanitas,41.946222,-87.649428,Mexican Restaurant,Moderate
9,5ac2cd2f35811b76aae32b5f,LAKE VIEW,41.947370,-87.654465,Big Star,41.948286,-87.657419,Taco Place,Cheap


In [75]:
Chicago_Neigh_Top_List.sort_values(['Income_&_Hispanic_Pop_Combined_Rank'],ascending=True).inplace=True
Chicago_Neigh_Top_List

,COMMUNITY NAME,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,Total Population,Hispanic or Latino,Median Age,Total Households,Male: 15 to 70 yrs,Female: 15 to 70 yrs,Not Hispanic or Latino,Hispanic or Lation % of Total,Latitude,Longitude,Income_Rank,H&L_Pop%_Rank,Age_Rank,Income_&_Hispanic_Pop_Combined_Rank
17,CLEARING,37.6,25113,23139.0,10484.0,35.540070,8467.0,8090.0,8230.0,12655.0,45.308786,41.775271,-87.751894,28,20,43,48
25,FOREST GLEN,40.5,44164,18508.0,2126.0,42.522330,6864.0,6058.0,6405.0,16382.0,11.486925,41.990214,-87.733479,8,39,20,47
28,GARFIELD RIDGE,38.1,26353,34513.0,13515.0,37.743557,11927.0,11916.0,11875.0,20998.0,39.159157,41.785894,-87.738031,25,22,39,47
35,IRVING PARK,31.6,27249,53359.0,24332.0,33.276580,20376.0,20254.0,19653.0,29027.0,45.600555,41.959528,-87.698606,22,19,59,41
38,LAKE VIEW,17.0,60058,94368.0,7198.0,30.411351,54425.0,41137.0,40661.0,87170.0,7.627586,41.947370,-87.654465,4,45,76,49
40,LINCOLN SQUARE,25.5,37524,39493.0,7562.0,33.450642,18194.0,15451.0,16002.0,31931.0,19.147697,41.968913,-87.674871,13,33,68,46
41,LOGAN SQUARE,26.2,31908,72791.0,37295.0,30.710988,29352.0,28987.0,27991.0,35496.0,51.235730,41.917349,-87.666405,19,16,66,35
50,NEAR WEST SIDE,22.2,44689,54881.0,5048.0,30.608212,26298.0,22789.0,23302.0,49833.0,9.198083,41.881643,-87.647206,7,41,71,48
52,NORTH CENTER,26.2,57123,31867.0,4330.0,32.478928,14561.0,12172.0,12570.0,27537.0,13.587724,41.954342,-87.674439,6,37,66,43
76,WEST TOWN,21.7,43198,82236.0,23901.0,31.257042,38190.0,34955.0,32724.0,58335.0,29.063914,41.899386,-87.696928,9,26,74,35


In [121]:
Chicago_Mex_Final = pd.merge(Community_Mex_Rest, Chicago_Neigh_Top_List, on='COMMUNITY NAME', how='outer')
# Chicago_Mex_Final.dtypes
Chicago_Mex_Final.sort_values(by=['Income_&_Hispanic_Pop_Combined_Rank'], ascending=False).inplace=True
Chicago_Mex_Final.drop(['id','Male: 15 to 70 yrs','Female: 15 to 70 yrs','Latitude','Longitude','Median Age','Age_Rank','Total Households'],axis = 1, inplace = True)
Chicago_Mex_Final

,COMMUNITY NAME,COMMUNITY Latitude,COMMUNITY Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Res_Tier,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,Total Population,Hispanic or Latino,Not Hispanic or Latino,Hispanic or Lation % of Total,Income_Rank,H&L_Pop%_Rank,Income_&_Hispanic_Pop_Combined_Rank
0,GARFIELD RIDGE,41.785894,-87.738031,Salgados Grocery,41.786266,-87.734107,Mexican Restaurant,Cheap,38.1,26353,34513.0,13515.0,20998.0,39.159157,25,22,47
1,GARFIELD RIDGE,41.785894,-87.738031,DeColores,41.788128,-87.742233,Mexican Restaurant,Cheap,38.1,26353,34513.0,13515.0,20998.0,39.159157,25,22,47
2,GARFIELD RIDGE,41.785894,-87.738031,Des Colores,41.788956,-87.742392,Burrito Place,Cheap,38.1,26353,34513.0,13515.0,20998.0,39.159157,25,22,47
3,IRVING PARK,41.959528,-87.698606,Que Rico,41.959259,-87.699812,Mexican Restaurant,Cheap,31.6,27249,53359.0,24332.0,29027.0,45.600555,22,19,41
4,IRVING PARK,41.959528,-87.698606,Emma's Fruits @ Horner Park,41.959496,-87.698189,Mexican Restaurant,Cheap,31.6,27249,53359.0,24332.0,29027.0,45.600555,22,19,41
5,LAKE VIEW,41.947370,-87.654465,Taco & Burrito Palace,41.945653,-87.649464,Mexican Restaurant,Cheap,17.0,60058,94368.0,7198.0,87170.0,7.627586,4,45,49
6,LAKE VIEW,41.947370,-87.654465,El Burrito Mexicano,41.947361,-87.653407,Mexican Restaurant,Cheap,17.0,60058,94368.0,7198.0,87170.0,7.627586,4,45,49
7,LAKE VIEW,41.947370,-87.654465,Salsita,41.949460,-87.649637,Mexican Restaurant,Cheap,17.0,60058,94368.0,7198.0,87170.0,7.627586,4,45,49
8,LAKE VIEW,41.947370,-87.654465,Las Mañanitas,41.946222,-87.649428,Mexican Restaurant,Moderate,17.0,60058,94368.0,7198.0,87170.0,7.627586,4,45,49
9,LAKE VIEW,41.947370,-87.654465,Big Star,41.948286,-87.657419,Taco Place,Cheap,17.0,60058,94368.0,7198.0,87170.0,7.627586,4,45,49


In [122]:
Chicago_Mex_Final['Income_&_Hispanic_Pop_Combined_Rank'].sort_values().inplace=True
Chicago_Mex_Final['Rank'] = Chicago_Mex_Final['Income_&_Hispanic_Pop_Combined_Rank'].rank(ascending = 1) 
Chicago_Mex_Final

,COMMUNITY NAME,COMMUNITY Latitude,COMMUNITY Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Res_Tier,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,Total Population,Hispanic or Latino,Not Hispanic or Latino,Hispanic or Lation % of Total,Income_Rank,H&L_Pop%_Rank,Income_&_Hispanic_Pop_Combined_Rank,Rank
0,GARFIELD RIDGE,41.785894,-87.738031,Salgados Grocery,41.786266,-87.734107,Mexican Restaurant,Cheap,38.1,26353,34513.0,13515.0,20998.0,39.159157,25,22,47,17.5
1,GARFIELD RIDGE,41.785894,-87.738031,DeColores,41.788128,-87.742233,Mexican Restaurant,Cheap,38.1,26353,34513.0,13515.0,20998.0,39.159157,25,22,47,17.5
2,GARFIELD RIDGE,41.785894,-87.738031,Des Colores,41.788956,-87.742392,Burrito Place,Cheap,38.1,26353,34513.0,13515.0,20998.0,39.159157,25,22,47,17.5
3,IRVING PARK,41.959528,-87.698606,Que Rico,41.959259,-87.699812,Mexican Restaurant,Cheap,31.6,27249,53359.0,24332.0,29027.0,45.600555,22,19,41,5.5
4,IRVING PARK,41.959528,-87.698606,Emma's Fruits @ Horner Park,41.959496,-87.698189,Mexican Restaurant,Cheap,31.6,27249,53359.0,24332.0,29027.0,45.600555,22,19,41,5.5
5,LAKE VIEW,41.947370,-87.654465,Taco & Burrito Palace,41.945653,-87.649464,Mexican Restaurant,Cheap,17.0,60058,94368.0,7198.0,87170.0,7.627586,4,45,49,28.0
6,LAKE VIEW,41.947370,-87.654465,El Burrito Mexicano,41.947361,-87.653407,Mexican Restaurant,Cheap,17.0,60058,94368.0,7198.0,87170.0,7.627586,4,45,49,28.0
7,LAKE VIEW,41.947370,-87.654465,Salsita,41.949460,-87.649637,Mexican Restaurant,Cheap,17.0,60058,94368.0,7198.0,87170.0,7.627586,4,45,49,28.0
8,LAKE VIEW,41.947370,-87.654465,Las Mañanitas,41.946222,-87.649428,Mexican Restaurant,Moderate,17.0,60058,94368.0,7198.0,87170.0,7.627586,4,45,49,28.0
9,LAKE VIEW,41.947370,-87.654465,Big Star,41.948286,-87.657419,Taco Place,Cheap,17.0,60058,94368.0,7198.0,87170.0,7.627586,4,45,49,28.0


In [123]:
indexNames = Chicago_Mex_Final[(Chicago_Mex_Final["COMMUNITY NAME"] == "CLEARING")].index
Chicago_Mex_Final.drop(indexNames , inplace=True)
indexNames = Chicago_Mex_Final[(Chicago_Mex_Final["COMMUNITY NAME"] == "FOREST GLEN")].index
Chicago_Mex_Final.drop(indexNames , inplace=True)

In [124]:
latitude = 41.961451
longitude = -87.6298
map_chicago_mex = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, community in zip(Chicago_Mex_Final['COMMUNITY Latitude'], Chicago_Mex_Final['COMMUNITY Longitude'], Chicago_Mex_Final['COMMUNITY NAME']):
    label = '{}'.format(community)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago_mex)  

map_chicago_mex

In [127]:
for lat, lng, community,income_rank,hispanic_pop_pct in zip(Chicago_Mex_Final['Venue Latitude'], Chicago_Mex_Final['Venue Longitude'], Chicago_Mex_Final['Venue'],Chicago_Mex_Final['Income_Rank'],Chicago_Mex_Final['Hispanic or Lation % of Total']):
    label = '{}'.format(community,income_rank,hispanic_pop_pct)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=False,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago_mex)  

map_chicago_mex

# Final Results:
There are only two communities that best fit the need:

1) West Town

2) Logan Square

### Reasons are:

1) There is good Hispanic and Latino Population

2) There is good income levels that can help start a moderate price range restuarant

3) There is good age range that eats outside